In [ ]:
"""
1. Search for an open opportunity at your possible potential clients
2. Search for relevant information from the opening that might act as a good starting point for your business with this company
3. Go through all of your company's projects and find relevant projects
4. Get important to share information about these projects
5. Write a compelling cold email about how your firm can be of great service to your potential client
6. Repeat for ALL potential clients
"""

In [ ]:
# Web Scraping

# Relevant information

# Chromadb query

# Email Generation

In [1]:
# Web Scraping
!pipenv install langchain_community

Loading .env environment variables...
Installing langchain_community...
Resolving langchain_community...
Added langchain-community to Pipfile's [packages] ...
✔ Installation Succeededain_community...
⠋ Installing langchain_community...
Pipfile.lock (f6673e) out of date, updating to (36fc99)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠏ Locking...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (e47eb3faf2be684b9139b2b625984300cea045c343a0c8f7369ffab3ac36fc99)!
Installing dependencies from Pipfile.lock (36fc99)...


In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://meinestadt.jobs.personio.de/job/1757093?language=en&display=de")
page_data = loader.load().pop().page_content
#print(page_data)

In [10]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

#response = llm.invoke("Who is the best footballer of all times?, no preamble")
#print(response.content)

In [16]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    I will give you scraped text from the job posting. 
    Your job is to extract the job details & requirements in a JSON format containing the following keys: 'role', 'experience', 'skills', and 'description'. 
    Only return valid JSON. No preamble, please.
    Here is the scraped text: {page_data}
    """
)

chain_extract = prompt_extract | llm 
response = chain_extract.invoke(input={'page_data':page_data})
print(type(response.content))
print(response.content)

<class 'str'>
```
{
  "role": "Team Lead Software Engineering",
  "experience": "Several years of software engineering experience coupled with more than 2 years in leadership roles",
  "skills": [
    "Solid technical expertise in designing and developing large-scale, end-to-end software solutions",
    "Approach complex issues with an open mindset and a focus on achieving simplicity",
    "Passionate about people, offering support and guidance to your engineers while avoiding micromanagement",
    "Skilled at rapidly establishing relationships with stakeholders and maintaining a solution-focused approach",
    "Capable of building high-performing, self-sufficient teams that effectively balance speed, quality, and risk"
  ],
  "description": "You will motivate, supervise and guide a software development team in disciplinary and technical terms. Establish personalized development or improvement plans, hold regular one-on-one meetings with team members, and cultivate a feedback-rich, non

In [17]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_response = json_parser.parse(response.content)
print(type(json_response))

<class 'dict'>


In [18]:
import csv

def read_csv_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)
        for row in csv_reader:
            # Separate technical skills (list) and project link (string)
            skills = tuple(row[:-1])  # Exclude the last element (project link)
            project_link = row[-1]  # Get the last element (project link)
            data.append((skills, project_link))  # Create a tuple with skills and link
    return data

# Example usage:
file_path = 'sample_portfolio.csv'  # Replace with your actual file path
data = read_csv_file(file_path)

for skills, project_link in data:
    print(skills, project_link)
    #print(project_link)


('Python', ' SQL', ' Pandas')  https://github.com/user/project1
('SQL', ' Python', ' Airflow')  https://github.com/user/project2
('PySpark', ' Spark SQL', ' Delta Lake')  https://github.com/user/project3
('Machine Learning', ' Deep Learning', ' TensorFlow')  https://github.com/user/project4
('Data Engineering', ' ETL', ' ELT')  https://github.com/user/project5
('Cloud Platforms (AWS', ' GCP', ' Azure)')  https://github.com/user/project6
('Data Warehousing', ' Data Modeling', ' DBT')  https://github.com/user/project7
('Data Visualization', ' Power BI', ' Tableau')  https://github.com/user/project8
('MLOps', ' MLflow', ' Kubeflow')  https://github.com/user/project9
('Natural Language Processing (NLP)', ' NLTK', ' spaCy')  https://github.com/user/project10
('Computer Vision', ' OpenCV', ' TensorFlow')  https://github.com/user/project11
('Time Series Analysis', ' Forecasting', ' Prophet')  https://github.com/user/project12
('Data Cleaning', ' Data Wrangling', ' Pandas')  https://github.com

In [21]:
# Insert data into Vector Database

import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio_links')

if not collection.count():
    for skills, project_url in data:
        collection.add(
            documents=str(skills),
            metadatas={'portfolio_url':project_url},
            ids=[str(uuid.uuid4())]
        )

In [39]:
json_response['skills']

['Solid technical expertise in designing and developing large-scale, end-to-end software solutions',
 'Approach complex issues with an open mindset and a focus on achieving simplicity',
 'Passionate about people, offering support and guidance to your engineers while avoiding micromanagement',
 'Skilled at rapidly establishing relationships with stakeholders and maintaining a solution-focused approach',
 'Capable of building high-performing, self-sufficient teams that effectively balance speed, quality, and risk']

In [40]:
portfolio_urls = collection.query(query_texts=json_response['skills'], n_results=2)
portfolio_urls

{'ids': [['b439c653-dc31-4c77-bca0-194e24a10f26',
   '1d98088e-54af-49fa-9f04-75dec617b791'],
  ['463e7751-bc4c-4a5e-ba26-81019b6ba6db',
   'b439c653-dc31-4c77-bca0-194e24a10f26'],
  ['b439c653-dc31-4c77-bca0-194e24a10f26',
   '66bed47a-a4a0-4fea-8322-cf715f345721'],
  ['2e476477-4ef5-45ea-8c43-c59ac3bee24c',
   'b439c653-dc31-4c77-bca0-194e24a10f26'],
  ['2e476477-4ef5-45ea-8c43-c59ac3bee24c',
   '66bed47a-a4a0-4fea-8322-cf715f345721']],
 'embeddings': None,
 'documents': [["('Feature Engineering', ' Scikit-learn')",
   "('Cloud Data Engineering', ' AWS Glue', ' Databricks')"],
  ["('Data Ethics', ' Privacy', ' GDPR')",
   "('Feature Engineering', ' Scikit-learn')"],
  ["('Feature Engineering', ' Scikit-learn')",
   "('Data Engineering', ' ETL', ' ELT')"],
  ["('Data Governance', ' Data Quality', ' Metadata Management')",
   "('Feature Engineering', ' Scikit-learn')"],
  ["('Data Governance', ' Data Quality', ' Metadata Management')",
   "('Data Engineering', ' ETL', ' ELT')"]],
 'uri

In [37]:
job_description = json_response['description']

In [41]:
email_prompt = PromptTemplate.from_template(
    """
    I will give you a role and a task that you have to perform in that specific role.
    Your Role: Your name is Hassan, You are an incredible business development officer who knows how to get clients. You work for X Consulting firm, your firm works with all sorts of IT clients and provide solutions in the domain of Data Science and AI. 
    X AI focuses on efficient tailored solutions for all clients keeping costs down. 
    Your Job: Your Job is to write cold emails to clients regarding the Job openings that they have advertised. Try to pitch your clients with an email hook that opens a conversation about a possibility of working with them. Add the most relevant portfolio URLs from
    the following (shared below) to showcase that we have the right expertise to get the job done. 
    I will now provide you with the Job description and the portfolio URLs:
    JOB DESCRIPTION: {job_description}
    ------
    PORTFOLIO URLS: {portfolio_urls}
    """
)

chain_email = email_prompt | llm
response = chain_email.invoke({'job_description':job_description, 'portfolio_urls':portfolio_urls})
print(response.content)

Subject: Expert Guidance for Your Software Development Team

Dear [Hiring Manager's Name],

I came across your job posting for a Team Lead/Manager to supervise and guide a software development team. As a Business Development Officer at X Consulting firm, I was impressed by your company's commitment to innovation and excellence.

At X Consulting, we specialize in providing tailored Data Science and AI solutions that drive business growth while keeping costs down. Our team of experts has a proven track record of delivering high-quality projects on time. I'd like to highlight a few examples of our work that demonstrate our expertise in software development and team management:

* Our project on Feature Engineering using Scikit-learn showcases our ability to develop and implement efficient software solutions. You can find more details here: https://github.com/user/project14
* Another project that demonstrates our expertise in Data Engineering using ETL and ELT can be found here: https://gi